In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
from utils.joystick_data_loader import JoystickDataModule
from utils.joystick_pressure_model import JoystickPressureModel

In [2]:
seq_len = 1000

In [3]:
model = JoystickPressureModel.load_from_checkpoint(checkpoint_path="logs/est_pressure_model/version_1/checkpoints/epoch=4-step=304775.ckpt", hidden_size=128)
trainer = pl.Trainer(accelerator='cuda')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
data = JoystickDataModule(dataset_path='result_peak_data.pkl', seq_len=seq_len, forecast_distance=2800*24,
                          batch_size=1000, n_of_worker=16, test_data_port_n=0)

In [6]:
trainer.test(model, datamodule=data)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       pressure_loss       │    0.5014541745185852     │
│         test_loss         │    0.7653132677078247     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.7653132677078247, 'pressure_loss': 0.5014541745185852}]

In [4]:
test_data = JoystickDataModule(dataset_path='result_peak_data.pkl', seq_len=seq_len, forecast_distance=2800*24,
                          batch_size=1000, n_of_worker=16, test_data_port_n=1)

In [5]:
trainer.test(model, datamodule=test_data)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



ValueError: 2 columns passed, passed data had 1000 columns

In [5]:
model.to_onnx('model.onnx', input_sample=torch.randn(1, 1000, 2), export_params=True)